`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 05.002

### 문제에서 주어진 변수

#### 각 구간 길이

In [ ]:
L_AB_m = 10.0
L_BD_m = 4.0

L_AD_m = L_AB_m + L_BD_m

#### 하중

In [ ]:
w_AB_N_m = -400e3
P_D_N = -1000e3

#### 단면

In [ ]:
tb_m = 150e-3
tt_m = 20e-3
ih_m = 200e-3

### A, B, C, D 지점의 x 좌표
A 점에서 x=0 으로 한다.

In [ ]:
x_A_m = 0
x_B_m = x_A_m + L_AB_m
x_D_m = x_B_m + L_BD_m

### 반력

A점 중심의 모멘트는 분포하중, D점 하중, E점 반력이 각각 가하는 모멘트의 합이 된다.

$$
\int_{x=0}^{x=L_{AB}} wx \, dx+R_B L_{AB}+P_D L_{AD}=0\\
$$

In [ ]:
x, L_AB, w, R_B, P_D, L_AD = sy.symbols('x, L_AB, w, R_B, P_D, L_AD')

나중에 사용하기 위해 변수값을 표 형태로 지정한다.

In [ ]:
s_d = {L_AB: L_AB_m, 
       w: w_AB_N_m, 
       P_D: P_D_N, 
       L_AD: L_AD_m}
s_d

In [ ]:
moment_eq = sy.Eq(sy.integrate(w * x, (x, 0, L_AB)) 
              + R_B * L_AB + P_D * L_AD,
              0)
moment_eq

B점 반력을 구하면 다음과 같다.

In [ ]:
R_B_sol_list = sy.solve(moment_eq, R_B)
R_B_sol_list

B점에서의 반력에 변수값을 대입한 결과는 다음과 같다.

In [ ]:
R_B_N = float(R_B_sol_list[0].subs(s_d))
R_B_N

A점에서의 반력은 수직방향 힘 평형 방정식으로부터 구할 수 있다.

A 점에서의 반력
$$
R_A + w L_{AB} + R_B + P_D = 0 \\
R_A = -w L_{AB} - R_B - P_D
$$

In [ ]:
R_A_N = - w_AB_N_m * L_AB_m - P_D_N - R_B_N
R_A_N

### 전단력선도

#### A-B 구간

In [ ]:
x_AB_m_array = np.linspace(x_A_m, x_B_m, 100+1)

A 지점에서의 반력과 분포하중을 반영한다.

$$V_{AB}=R_A+wx$$

In [ ]:
V_AB_N_array = R_A_N + w_AB_N_m * x_AB_m_array
V_AB_N_array[:5], V_AB_N_array[-5:], 

In [ ]:
plt.clf()
plt.fill_between(x_AB_m_array, V_AB_N_array)
plt.ylabel('V(N)')

A 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax = plt.axes()
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_A_N), fc='k', ec='k')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.xlim(xmax=x_D_m)
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

#### B-D 구간

In [ ]:
x_BD_m_array = np.linspace(x_B_m, x_D_m, 40+1)

A 지점에서의 반력과 분포하중, B 지점 반력을 반영한다.

$$V_{BD}=R_A+wL_{AB}+R_B$$

In [ ]:
V_BD_N_array = (R_A_N + w_AB_N_m * L_AB_m + R_B_N) * np.ones_like(x_BD_m_array)

In [ ]:
plt.clf()
plt.fill_between(x_AB_m_array, V_AB_N_array)
plt.fill_between(x_BD_m_array, V_BD_N_array)
plt.ylabel('V(N)')

A 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax = plt.axes()
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.2, head_length=0.05 * abs(R_A_N), fc='k', ec='k')

B 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax.arrow(x_B_m, V_AB_N_array[-1], 
         0, R_B_N * 0.9, 
         head_width=0.2, head_length=0.1 * abs(R_B_N), fc='k', ec='k')

D 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax.arrow(x_D_m, V_BD_N_array[-1], 
         0, P_D_N * 0.9, 
         head_width=0.2, head_length=0.1 * abs(P_D_N), fc='k', ec='k')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()